In [1]:
import sys

In [2]:
sys.path.insert(1, '../')

In [3]:
from utils.dataset import EHRDataset
from model.tokenizer import EHRTokenizer
import pytorch_pretrained_bert as Bert
from torch.utils.data import DataLoader
from model.model import *
from utils.config import *
from utils.optimizer import adam
from model.trainers import TrainerMLM
import pytorch_lightning as pl
from sklearn.model_selection import KFold
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
#from pytorch_lightning.strategies import DeepSpeedStrategy
warnings.filterwarnings('ignore')

from utils.vocabulary import *

In [4]:
pl.__version__

'1.5.10'

In [5]:
#path = 'C:/Users/Johan/Documents/Skola/MasterThesis/Master-thesis/pre-processing/combined-csv-files.csv'
path_patients = '../data/datasets/readmission_data_synthea'
path_prior = '../data/datasets/prior_table'

In [6]:
datapat = pd.read_parquet(path_patients)
dataprior = pd.read_parquet(path_prior)

In [7]:
dataset = datapat.merge(dataprior, on='subject_id', how='inner')

In [8]:
dataset

,subject_id,label,gender,diagnos_code,medication_code,age,hadm_id,prior_values,prior_indicies
0,001b1739-594d-5546-df1a-31b45f21df14,"[True, False, True, False, False, False, False...",F,"[[59621000], [160903007], [160903007, 73595000...","[[895994, 2123111, 308136], [895994, 2123111, ...","[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 27, 2...","[c6a1d752-8c9d-881e-7364-638199b36886, 12ffc33...","[[0.05721096694469452, 0.2824791371822357, 0.0...","[[59621000,2123111, 59621000,308136, 59621000,..."
1,0021064f-1f2d-c10d-cd67-61355163e451,"[True, False]",M,"[[59621000, 224299000, 160903007, 423315002, 7...","[[308136], [310798, 308136], [310798, 308136]]","[18, 19, 20]","[fd9765ce-323b-788d-e718-b12d381221ae, 183786f...","[[0.2824791371822357, 0.43951985239982605, 0.2...","[[59621000,308136, 423315002,308136, 224299000..."
2,00321cbc-4a3c-74db-2647-227bb48c9030,"[True, False, True, False, False, True, False,...",F,"[[59621000, 5251000175109, 713458007, 73595000...","[[314076], [314076, 308136], [314076, 308136],...","[19, 20, 21, 22, 23, 23, 23, 24, 25, 26, 27, 2...","[bd7bfe04-d279-4619-20d5-a8e1ed60706e, 280ff45...","[[0.576877236366272, 0.6760563254356384, 0.638...","[[59621000,314076, 713458007,314076, 73595000,..."
3,00357297-ebcc-09ad-ff09-44ce4edb2903,"[False, False, False, False, False, False, Fal...",F,"[[59621000, 105531004, 224299000], [706893006]...","[[314076], [314076], [314076], [314076], [3140...","[18, 19, 23, 24, 26, 27, 30, 44, 45, 48, 49, 5...","[3586f8d3-eb86-34b4-5126-f9de2b738144, 9c93df7...","[[0.576877236366272, 0.5573770403862, 0.572481...","[[59621000,314076, 105531004,314076, 224299000..."
4,005c8e32-1655-350c-4ee0-421e1f5bd9f5,"[True, False, False, False, True]",F,"[[59621000, 5251000175109, 73438004], [4085120...","[[310798], [310798, 314076], [310798, 314076],...","[18, 19, 56, 58, 59]","[93b22e64-503e-36d7-73c9-e1e7c6bde553, bf89c08...","[[0.4481525719165802, 0.4363636374473572, 0.46...","[[59621000,310798, 73438004,310798, 5251000175..."
...,...,...,...,...,...,...,...,...,...
903,5678bdb0-e556-b3c4-ea2b-ca4e841326ec,"[True, True, True]",M,"[[10509002], [160904001], [160903007, 706893006]]","[[313782], [200033, 313988, 314077], [200033, ...","[52, 52, 54]","[9d6b74e3-8080-5455-11ed-2ff202d31ef5, 77c8934...","[[0.7518796920776367, 0.6944444179534912], [0....","[[10509002,313782, 313782,10509002], [16090400..."
904,56963914-5b48-1103-2313-a21e76a27ba7,"[True, False, False, False, False, False, Fals...",M,"[[59621000, 224299000], [73595000], [706893006...","[[310798], [310798, 314076], [310798, 314076],...","[18, 19, 20, 21, 22, 27, 29, 31, 33, 37, 39, 4...","[d64182c8-8650-2a3c-d621-4091b5489f35, 76ba365...","[[0.4481525719165802, 0.4422604441642761, 0.02...","[[59621000,310798, 224299000,310798, 310798,59..."
905,56f6c873-86fc-869b-f314-d9c63e6f180a,"[False, False, False, False, False, False, Fal...",M,"[[59621000, 5251000175109], [160903007, 735950...","[[310798, 314076], [310798, 314076], [310798, ...","[18, 19, 21, 22, 23, 24, 25, 27, 28, 29, 30, 3...","[1fe98dd3-727c-3524-d021-db3565721970, fb6b0dd...","[[0.576877236366272, 0.4481525719165802, 0.563...","[[59621000,314076, 59621000,310798, 5251000175..."
906,5710b35c-5e42-d52b-e0de-df001aa6dadf,"[False, False, False, False, False, False, Fal...",M,"[[160968000], [59621000, 224299000, 160903007]...","[[895994, 2123111], [895994, 2123111, 314076],...","[17, 18, 20, 21, 22, 23, 24, 25, 26, 29, 30, 3...","[55b65f82-750a-13ab-6d94-4ab5835df1a5, df90e2f...","[[0.8333333134651184, 0.8333333134651184, 0.01...","[[160968000,2123111, 160968000,895994, 2123111..."


In [9]:
global_params = {
    'max_seq_len': 256,
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 32,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'device': 'cuda' #change this to run on cuda #'cuda:0'
}

In [10]:
dataset['subject_id'] = dataset['subject_id'].apply(lambda x: x.replace('-', ''))

In [11]:
cand1 = '03b2c53316751658bec6774dac0420e8'
cand2 = '530721d8beaad75b132645f864310ac5'
cand3 = '3c8ea5fab1efb6c3d7fb6b5e90d006db'

In [12]:
dataset[dataset['subject_id'] == cand1]['diagnos_code'].tolist()

[array([array([ 59621000, 224299000, 160903007, 424393004]),
        array([160903007,  73595000]), array([160903007]),
        array([38822007])], dtype=object)]

In [13]:
[ 59621000, 224299000, 160903007, 424393004, 310798, 'SEP', 160903007,  73595000, 310798, 'SEP', 160903007, 310798, 'SEP', 38822007, 311989, 1094107, 'SEP'][:6]

[59621000, 224299000, 160903007, 424393004, 310798, 'SEP']

In [14]:
sequence = [20, 21, 22, 22, 'SEP', 25, 25, 25, 'SEP', 28, 28, 28, 'SEP', 29, 29, 'SEP'][:10]

In [15]:
len(sequence) - sequence[::-1].index('SEP')

9

In [16]:
dataset[dataset['subject_id'] == cand1]['medication_code'].tolist()

[array([array([310798], dtype=int32), array([310798], dtype=int32),
        array([310798], dtype=int32),
        array([ 311989, 1094107], dtype=int32)], dtype=object)]

In [17]:
dataset[dataset['subject_id'] == cand1]['prior_values'].tolist()

[array([array([0.44815257, 0.60892391, 0.57646614, 0.44226044, 0.02893866,
               0.01785577, 0.70207036, 0.01385361])                       ,
        array([0.57646614, 0.57566994, 0.70207036, 0.22981605]),
        array([0.57646614, 0.70207036]),
        array([1.        , 1.        , 0.23529412, 0.23529412])],
       dtype=object)]

In [18]:
dataset[dataset['subject_id'] == cand1]['prior_indicies'].tolist()

[array([array(['59621000,310798', '424393004,310798', '160903007,310798',
               '224299000,310798', '310798,59621000', '310798,424393004',
               '310798,160903007', '310798,224299000'], dtype=object)    ,
        array(['160903007,310798', '73595000,310798', '310798,160903007',
               '310798,73595000'], dtype=object)                         ,
        array(['160903007,310798', '310798,160903007'], dtype=object),
        array(['38822007,1094107', '38822007,311989', '1094107,38822007',
               '311989,38822007'], dtype=object)                         ],
       dtype=object)]

In [19]:
#diagnos_codes = dataset['diagnos_code'].tolist()
#med_codes = dataset['medication_code'].tolist()
#diagnos_codes.extend(med_codes)
#file_path = '../data/vocabularies/Synthea/snomedrxnorm'
#write_codes_to_file(diagnos_codes, file_path)

In [20]:
#ages = dataset['age'].tolist()
#file_path = '../data/vocabularies/Synthea/age'
#write_age_to_file(ages, file_path)

In [21]:
files = {'code':'../data/vocabularies/Synthea/snomedrxnorm.npy',
         'age':'../data/vocabularies/Synthea/age.npy'
        }
tokenizer = EHRTokenizer(task='MLM', filenames=files)

In [22]:
len(tokenizer.getVoc('code').keys())

258

In [23]:
len(tokenizer.getVoc('age').keys())

104

In [24]:
#Shallow
model_config = {
    'vocab_size': len(tokenizer.getVoc('code').keys()), # number of disease + symbols for word embedding
    'hidden_size': 288, # word embedding and seg embedding hidden size
    'seg_vocab_size': 2, # number of vocab for seg embedding
    'age_vocab_size': len(tokenizer.getVoc('age').keys()), # number of vocab for age embedding,
    'gender_vocab_size': 3,
    'max_position_embeddings': train_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 4, # number of multi-head attention layers required
    'num_attention_heads': 6, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 512, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
}

In [25]:
conf = BertConfig(model_config)

In [26]:
tensorboarddir = '../logs/'

In [27]:
#kfold=KFold(n_splits=5,shuffle=True)

In [28]:
trainer = pl.Trainer(
            max_epochs=5, 
            gpus=-1,
            logger=pl.loggers.TensorBoardLogger(save_dir=tensorboarddir),
            callbacks=[pl.callbacks.TQDMProgressBar()], 
            progress_bar_refresh_rate=1,
            weights_summary=None, # Can be None, top or full
            num_sanity_val_steps=10,
        )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [29]:
trainset, testset = train_test_split(dataset, test_size=0.2)

In [30]:
torch.__version__

'1.10.2'

In [31]:
traind = EHRDataset(trainset, max_len=train_params['max_len_seq'], tokenizer=tokenizer)
testd = EHRDataset(testset, max_len=train_params['max_len_seq'], tokenizer=tokenizer)

In [32]:
trainloader = torch.utils.data.DataLoader(traind, batch_size=32, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testd, batch_size=32, shuffle=True, num_workers=4)

In [33]:
test = next(iter(trainloader))

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/stud/j/johagg17/.conda/envs/masterenv/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/stud/j/johagg17/.conda/envs/masterenv/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "/home/stud/j/johagg17/.conda/envs/masterenv/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 84, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/stud/j/johagg17/.conda/envs/masterenv/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 84, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/stud/j/johagg17/.conda/envs/masterenv/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 56, in default_

In [ ]:
model = BertForMaskedLM(conf) #BertForMaskedLM(conf)
params = list(model.named_parameters())
optim = adam(params, optim_param)

In [ ]:
patienttrajectory = TrainerMLM(model, optim, optim_param)

trainer.fit(
    patienttrajectory, 
    train_dataloaders=trainloader,
);

predictions = trainer.predict(patienttrajectory, dataloaders=testloader)

In [ ]:
avg_acc = sum([ stats['precision'] for stats in predictions ]) / len(predictions)
avg_acc*100

In [ ]:
PATH = '../saved_models/MLM/deep_notsuffled'
torch.save(model.state_dict(), PATH)

In [ ]:
# This is for the crossval, might not be runnable since we have done some debugging

'''
for fold,(train_idx,test_idx) in enumerate(kfold.split(data)):
    print('------------fold no---------{}----------------------'.format(fold))
    
    trainer = pl.Trainer(
            max_epochs=5, 
            gpus=-1,
            logger=pl.loggers.TensorBoardLogger(save_dir=tensorboarddir),
            callbacks=[pl.callbacks.progress.TQDMProgressBar()], 
            progress_bar_refresh_rate=1,
            weights_summary=None, # Can be None, top or full
            num_sanity_val_steps=10,
        )
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)
    
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=train_params['batch_size'], sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(dataset, batch_size=train_params['batch_size'], sampler=test_subsampler)
    
    model = BertForMaskedLM(conf) #BertForMaskedLM(conf)
    params = list(model.named_parameters())
    optim = adam(params, optim_param)
    
    patienttrajectory = PatientTrajectoryPredictor(model, optim, optim_param, metrics=True)
    
    trainer.fit(
        patienttrajectory, 
        train_dataloader=trainloader,
    );
  
    predictions = trainer.predict(patienttrajectory, dataloaders=testloader)
    avg_acc = sum([ stats['Precision'] for stats in predictions ]) / len(predictions)
    
    if avg_acc > best_test:
        PATH = '../saved_models/Shallow_unsuffled{}'.format(fold)
        torch.save(model.state_dict(), PATH)
        best_test = avg_acc
        
    print("Average Test precision for fold {}: {} ".format(fold, avg_acc))
'''

In [ ]:
PATH = '../saved_models/Shallow_unsuffled{}'.format(fold)
torch.save(model.state_dict(), PATH)

In [ ]:
avg_acc

In [ ]:
np.mean(allacc)

### Testing with BEHRTs training approach (without pytorch lightning)

In [ ]:
#import sklearn.metrics as skm

In [ ]:
'''
def cal_acc(label, pred):
    logs = nn.LogSoftmax()
    label=label.cpu().numpy()
    ind = np.where(label!=-1)[0]
    truepred = pred.detach().cpu().numpy()
    truepred = truepred[ind]
    truelabel = label[ind]
    truepred = logs(torch.tensor(truepred))
    outs = [np.argmax(pred_x) for pred_x in truepred.numpy()]
    
   # print("Truelabel:")
   # print(truelabel)
    
   # print("Output:")
   # print(outs)
    precision = skm.precision_score(truelabel, outs, average='micro')
    return precision
'''

In [ ]:
#conf = BertConfig(model_config)
#model = BertForMaskedLM(conf) 
#model = model.to('cuda')

In [ ]:
#params = list(model.named_parameters())
#optim = adam(params, optim_param)

In [ ]:
'''
def train(e, loader):
    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt= 0
    start = time.time()

    for step, batch in enumerate(loader):
        cnt +=1
        batch = tuple(t.to(train_params['device']) for t in batch)
        age_ids, gender_ids, input_ids, posi_ids, segment_ids, attMask, masked_label, _ = batch
        #print()
        loss, pred, label = model(input_ids, age_ids, gender_ids, segment_ids, posi_ids,attention_mask=attMask, labels=masked_label)
        if global_params['gradient_accumulation_steps'] >1:
            loss = loss/global_params['gradient_accumulation_steps']
        loss.backward()
        
        temp_loss += loss.item()
        tr_loss += loss.item()
        
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        
        if step % 200==0:
            print("epoch: {}\t| cnt: {}\t|Loss: {}\t| precision: {:.4f}\t| time: {:.2f}".format(e, cnt, temp_loss/2000, cal_acc(label, pred), time.time()-start))
            temp_loss = 0
            start = time.time()
            
        if (step + 1) % global_params['gradient_accumulation_steps'] == 0:
            optim.step()
            optim.zero_grad()

   #print("** ** * Saving fine - tuned model ** ** * ")
    #model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
    #create_folder(file_config['model_path'])
    #output_model_file = os.path.join(file_config['model_path'], file_config['model_name'])

    #torch.save(model_to_save.state_dict(), output_model_file)
        
    cost = time.time() - start
    return tr_loss, cost
    
'''

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
trainset = EHRDataset(trainset, max_len=train_params['max_len_seq'], tokenizer=tokenizer)
trainload = DataLoader(dataset=trainset, batch_size=train_params['batch_size'], shuffle=True, num_workers=2)

In [ ]:
import time

In [ ]:
for e in range(100):
    loss, timecost = train(e, trainload)
    loss = loss / len(trainload)
    
    print("Loss after epoch {}: {}".format(e, loss))
    
    